In [42]:
import argparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [43]:
# if __name__ == '__main__':
#     # accepts commandline arguments

#     parser = argparse.ArgumentParser(description='Process train data and test data.')

#     # Add arguments
#     parser.add_argument('train_data', type=int, help='train dataset')
#     parser.add_argument('test_data', type=int, help='test dataset')

#     # Parse the arguments
#     args = parser.parse_args()

#     # Now you can use args.grid_id and args.n_components in your script
#     train_data = args.train_data
#     test_data = args.test_data

# Grid_id = 333519

In [44]:
!python3 pca_pipeline.py 333519 200 '../datasets/train/data_heatmap_train.csv'

pca_df = pd.read_csv('temp/pca_df.csv')
pca_df = pca_df.iloc[:,1:]
pca_df_inp = pca_df.iloc[:,:-2]
y_train = pca_df['label'].values

/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
^C
Traceback (most recent call last):
  File "/Users/yinyue/Documents/Graduate/ENGI4800 Capstone/broken-banner-detection/notebooks/pca_pipeline.py", line 6, in <module>
    import seaborn as sns
  File "/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/seaborn/__init__.py", line 12, in <module>
    from .widgets import *  # noqa: F401,F403
  File "/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/seaborn/widgets.py", line 7, in <module>
    from ipywidgets import interact, FloatSlider, IntSlider
  File "/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/ipywidgets/__init__.py", line 25, in <module>
    from .widgets import *
  File "/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/ipywidgets/widgets/_

In [ ]:
!python3 pca_pipeline.py 333519 200 '../datasets/test/data_heatmap_test.csv'

pca_test = pd.read_csv('temp/pca_df.csv')
pca_test = pca_test.iloc[:,1:]
y_test = pca_test[['label']].values

/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Grid ID:  333519
Nb components:  200
Data directory:  ../datasets/test/data_heatmap_test.csv


## K-means

### NEW APPROACH

In [64]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import euclidean_distances
from sklearn.metrics import pairwise_distances_argmin_min

# Standardize the data
scaler = StandardScaler()
data_normalized = scaler.fit_transform(pca_df_inp)

kmeans = KMeans(n_clusters=4, random_state=42).fit(data_normalized)
centroids = kmeans.cluster_centers_
labels = kmeans.labels_

# Calculate the distance from each point to its cluster centroid
closest, distances = pairwise_distances_argmin_min(data_normalized, centroids[labels])
# distances = euclidean_distances(data_normalized, centroids[labels])

# Choose a threshold for flagging an anomaly
threshold = np.percentile(distances, 95)

# # Anything above the threshold is considered an anomaly
anomalies = pca_df_inp[distances > threshold]

kmeans_labels = (distances > threshold).astype(int)
confusion_matrix(y_train,kmeans_labels)

array([[557,  13],
       [ 22,  18]])

In [ ]:
# kchoices = [1,2,3,4,5,6]

best_acc = -float('inf')
best_dim = -1
best_recall = -1
best_prec = -1
best_f1 = -1

# for k in kchoices:
for pca_dim in np.linspace(2,200,50).astype(int):
      X = pca_df_inp.iloc[:,:pca_dim]
      kmeans = KMeans(n_clusters = 2, random_state=42)
      kmeans_labels = kmeans.fit_predict(X)
      # Identify the smallest cluster or the one furthest from others
      # cluster_minority = pd.Series(clusters).value_counts()
      # cluster_minority = cluster_minority[cluster_minority <= 60].index.get_level_values(0).tolist()
      # kmeans_labels = np.isin(clusters, cluster_minority).astype(int)
      conf_mat   = confusion_matrix(y_train,kmeans_labels)
      acc        = accuracy_score(y_train,kmeans_labels)
      recall     = recall_score(y_train,kmeans_labels)
      prec       = precision_score(y_train,kmeans_labels)
      f1         = f1_score(y_train,kmeans_labels)
      
      if f1 > best_f1:
            best_acc = acc
            best_dim = pca_dim
            best_recall  = recall
            best_prec = prec
            best_f1 = f1
            # best_k = k
            # print(f'Confusion matrix for {pca_dim} PCA dimensions:\n', 
            #       f' {conf_mat}')
            # print(f'Accuracy for {pca_dim} PCA dimensions:', 
            #       f' {acc:.2f}')
            # print(f'Recall for {pca_dim} PCA dimensions:', 
            #       f' {recall:.2f}')
            # print(f'Precision for {pca_dim} PCA dimensions:', 
            #       f' {prec:.2f}')
            # print(f'---------')
#     print(f'-------------------------------------')
# print(f'Best k: {best_k}')

print(f'Best PCA dimension: {best_dim}')
print(f'Best accuracy: {best_acc:.2f}')
print(f'Best precision: {best_prec:.2f}')
print(f'Best recall: {best_recall:.2f}')
print(f'Best f1-score: {best_f1:.2f}')

Best PCA dimension: 86
Best accuracy: 0.99
Best precision: 0.89
Best recall: 0.97
Best f1-score: 0.93


In [ ]:
X = pca_df_inp.iloc[:,:best_dim]
kmeans = KMeans(n_clusters = 2, random_state=42)
kmeans.fit(X)
# clusters = pd.DataFrame(kmeans.labels_)
# Identify the smallest cluster or the one furthest from others
# cluster_minority = clusters.value_counts()
# cluster_minority = cluster_minority[cluster_minority <= 60].index.get_level_values(0).tolist()
kmeans_labels = kmeans.predict(pca_test.iloc[:,:best_dim])
# kmeans_labels = np.isin(pred, cluster_minority).astype(int)
conf_mat = confusion_matrix(y_test,kmeans_labels)
acc = accuracy_score(y_test,kmeans_labels)
recall = recall_score(y_test,kmeans_labels)
prec = precision_score(y_test,kmeans_labels)
f1 = f1_score(y_test,kmeans_labels)
print(conf_mat)
print(f'Test accuracy: {acc:.2f}')
print(f'Test precision: {prec:.2f}')
print(f'Test recall: {recall:.2f}')
print(f'Test f1-score: {f1:.2f}')

[[241   4]
 [  2  15]]
Test accuracy: 0.98
Test precision: 0.79
Test recall: 0.88
Test f1-score: 0.83


## KNN

In [ ]:
# Perform PCA on X_train and X_test with 5 dimensions
X_train = pca_df_inp.iloc[:, :5]  
X_test = pca_test.iloc[:, :5] 

# Train k-NN model on X_train_pca with 10 neighbors
knn = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')
knn.fit(X_train)

# Predict anomalies on X_test_pca
distances, indices = knn.kneighbors(X_test)
anomaly_scores = distances.mean(axis=1)
threshold = np.percentile(anomaly_scores, 90)

# Convert labels to binary (0: inliers, 1: outliers)
knn_labels = np.where(anomaly_scores > threshold, 1, 0)

# Evaluate the performance on X_test
conf_mat = confusion_matrix(y_test, knn_labels)
acc = accuracy_score(y_test, knn_labels)
f1 = f1_score(y_test, knn_labels)

print(f'Confusion matrix on X_test:\n{conf_mat}')
print(f'Accuracy on X_test: {acc:.2f}')
print(f'F1 Score on X_test: {f1:.2f}')
print("Predicted knn labels:")
print(knn_labels)

Confusion matrix on X_test:
[[233  12]
 [  2  15]]
Accuracy on X_test: 0.95
F1 Score on X_test: 0.68
Predicted knn labels:
[0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0]


## DBSCAN

In [ ]:
# Apply PCA with 8 dimensions
X_train = pca_df_inp.iloc[:, :8]  
X_test = pca_test.iloc[:, :8] 

# Fit DBSCAN model
dbscan = DBSCAN(eps=10, min_samples=10)
dbscan_labels_train = dbscan.fit_predict(X_train)
dbscan_labels_test = dbscan.fit_predict(X_test)

# Convert labels to binary (0: inliers, 1: outliers)
dbscan_labels_train = np.where(dbscan_labels_train >= 0, 0, 1)
dbscan_labels_test = np.where(dbscan_labels_test >= 0, 0, 1)

# Evaluate performance on test data
conf_mat_test = confusion_matrix(y_test, dbscan_labels_test)
acc_test = accuracy_score(y_test, dbscan_labels_test)
f1_test = f1_score(y_test, dbscan_labels_test)

print("\nPerformance on test data:")
print("Confusion matrix:")
print(conf_mat_test)
print("Accuracy:", acc_test)
print("F1 Score:", f1_test)
print("Predicted DBScan labels:")
print(dbscan_labels_test)


Performance on test data:
Confusion matrix:
[[235  10]
 [  0  17]]
Accuracy: 0.9618320610687023
F1 Score: 0.7727272727272727
Predicted DBScan labels:
[0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0
 0 1 0]


## Isolation Forest

In [ ]:
#Best Model for Isolation Forest is with 10 PCA and n_estimators = 100
best_n_est = 100
pca_dim = 10
X_train = pca_df_inp.iloc[:,:pca_dim]
X_test = pca_test.iloc[:,:pca_dim]
            
isolation_forest = IsolationForest(n_estimators=best_n_est, 
                                   max_samples='auto', 
                                   random_state=42)

model = isolation_forest.fit(X_train)
if_labels = model.predict(X_test)
if_labels = np.where(if_labels == -1, 1, 0)

conf_mat   = confusion_matrix(y_test,if_labels)
acc        = accuracy_score(y_test,if_labels)
recall     = recall_score(y_test,if_labels)
prec       = precision_score(y_test,if_labels)
f1         = f1_score(y_test, if_labels)


print(f'Confusion matrix for {pca_dim} PCA dimensions:\n', 
                      f' {conf_mat}')
#print(f'Accuracy for {pca_dim} PCA dimensions:', 
#                      f' {acc:.2f}')
#print(f'Recall for {pca_dim} PCA dimensions:', 
#                      f' {recall:.2f}')
#print(f'Precision for {pca_dim} PCA dimensions:', 
#                      f' {prec:.2f}')
print(f'F1 Score for {pca_dim} PCA dimensions with n_estimator = {best_n_est}:', 
                      f' {f1:.2f}')
print(f'---------')

Confusion matrix for 10 PCA dimensions:
  [[224  21]
 [  1  16]]
F1 Score for 10 PCA dimensions with n_estimator = 100:  0.59
---------


## One-class SVM

In [ ]:
# Best Model is with 163 PCA dimensions
pca_dim = 163

X_train = pca_df_inp.iloc[:,:pca_dim]
X_test = pca_test.iloc[:,:pca_dim]

oneclass = OneClassSVM(gamma = 'auto').fit(X_train)
oneclass_labels = oneclass.predict(X_test)
    
oneclass_labels = np.where(oneclass_labels == 1, 0,1)
conf_mat   = confusion_matrix(y_test,oneclass_labels)
acc        = accuracy_score(y_test,oneclass_labels)
recall     = recall_score(y_test,oneclass_labels)
prec       = precision_score(y_test,oneclass_labels)
f1         = f1_score(y_test,oneclass_labels)

print(f'Confusion matrix for {pca_dim} PCA dimensions:\n', 
              f' {conf_mat}')
#print(f'Accuracy for {pca_dim} PCA dimensions:', 
#              f' {acc:.2f}')
#print(f'Recall for {pca_dim} PCA dimensions:', 
#              f' {recall:.2f}')
#print(f'Precision for {pca_dim} PCA dimensions:', 
#              f' {prec:.2f}')
print(f'F1 Score for {pca_dim} PCA dimensions:', 
              f' {f1:.2f}')

Confusion matrix for 163 PCA dimensions:
  [[ 57 188]
 [  0  17]]
F1 Score for 163 PCA dimensions:  0.15


## Combine

In [ ]:
best_acc = -float('inf')
best_dim = -1
best_recall = -1
best_prec = -1
best_f1 = -1

for i1 in [0,1]:
    for i2 in [0,1]:
        for i3 in [0,1]:
            for i4 in [0,1]:
                for i5 in [0,1]:
                    for thresh in [0,1]:
                        tot = i1+i2+i3+i4+i5
                        comb_labels = i1*if_labels + i2*oneclass_labels + i3*dbscan_labels_test + \
                                        i4*kmeans_labels + i5*knn_labels
                        comb_labels = np.where(comb_labels > tot//2 + thresh, 1, 0)

                        conf_mat   = confusion_matrix(y_test,comb_labels)
                        acc        = accuracy_score(y_test,comb_labels)
                        recall     = recall_score(y_test,comb_labels)
                        prec       = precision_score(y_test,comb_labels)
                        f1         = f1_score(y_test,comb_labels)

                        if acc > best_acc:
                            best_acc = acc
                            best_recall  = recall
                            best_prec = prec
                            best_conf_mat = conf_mat
                            best_f1 = f1
print(f'Confusion matrix for ({i1,i2,i3,i4,i5}) included and {thresh}:\n', f' {conf_mat}')
print(f'Best f1 score: {best_f1:.2f}')
                        #     print(f'Accuracy for ({i1,i2,i3,i4,i5}) included and {thresh}:', 
                        #           f' {acc:.2f}')
                        #     print(f'Recall for ({i1,i2,i3,i4,i5}) included and {thresh}:', 
                        #           f' {recall:.2f}')
                        #     print(f'Precision for ({i1,i2,i3,i4,i5}) included and {thresh}:', 
                        #           f' {prec:.2f}')

Confusion matrix for ((1, 1, 1, 1, 1)) included and 1:
  [[241   4]
 [  1  16]]
Best f1 score: 0.87


# Grid_id = 333346

In [ ]:
!python3 pca_pipeline.py 333346 200 '../datasets/train/data_heatmap_train.csv'

pca_df = pd.read_csv('temp/pca_df.csv')
pca_df = pca_df.iloc[:,1:]
pca_df_inp = pca_df.iloc[:,:-2]
y_train = pca_df[['label']].values

/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Grid ID:  333346
Nb components:  200
Data directory:  ../datasets/train/data_heatmap_train.csv


In [ ]:
!python3 pca_pipeline.py 333346 200 '../datasets/test/data_heatmap_test.csv'

pca_test = pd.read_csv('temp/pca_df.csv')
pca_test = pca_test.iloc[:,1:]
y_test = pca_test[['label']].values

/Users/yinyue/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Grid ID:  333346
Nb components:  200
Data directory:  ../datasets/test/data_heatmap_test.csv


## K-means

In [ ]:
# kchoices = [1,2,3,4,5,6]

best_acc = -float('inf')
best_dim = -1
best_recall = -1
best_prec = -1
best_f1 = -1

# for k in kchoices:
for pca_dim in np.linspace(2,200,50).astype(int):
      X = pca_df_inp.iloc[:,:pca_dim]
      kmeans = KMeans(n_clusters = 2, random_state=42)
      kmeans_labels = kmeans.fit_predict(X)
      # Identify the smallest cluster or the one furthest from others
      # cluster_minority = pd.Series(clusters).value_counts()
      # cluster_minority = cluster_minority[cluster_minority <= 60].index.get_level_values(0).tolist()
      # kmeans_labels = np.isin(clusters, cluster_minority).astype(int)
      conf_mat   = confusion_matrix(y_train,kmeans_labels)
      acc        = accuracy_score(y_train,kmeans_labels)
      recall     = recall_score(y_train,kmeans_labels)
      prec       = precision_score(y_train,kmeans_labels)
      f1         = f1_score(y_train,kmeans_labels)
      
      if f1 > best_f1:
            best_acc = acc
            best_dim = pca_dim
            best_recall  = recall
            best_prec = prec
            best_f1 = f1
            # best_k = k
            # print(f'Confusion matrix for {pca_dim} PCA dimensions:\n', 
            #       f' {conf_mat}')
            # print(f'Accuracy for {pca_dim} PCA dimensions:', 
            #       f' {acc:.2f}')
            # print(f'Recall for {pca_dim} PCA dimensions:', 
            #       f' {recall:.2f}')
            # print(f'Precision for {pca_dim} PCA dimensions:', 
            #       f' {prec:.2f}')
            # print(f'---------')
#     print(f'-------------------------------------')
# print(f'Best k: {best_k}')

print(f'Best PCA dimension: {best_dim}')
print(f'Best accuracy: {best_acc:.2f}')
print(f'Best precision: {best_prec:.2f}')
print(f'Best recall: {best_recall:.2f}')
print(f'Best f1-score: {best_f1:.2f}')

Best PCA dimension: 54
Best accuracy: 0.90
Best precision: 1.00
Best recall: 0.20
Best f1-score: 0.34


In [ ]:
X = pca_df_inp.iloc[:,:best_dim]
kmeans = KMeans(n_clusters = 2, random_state=42)
kmeans.fit(X)
# clusters = pd.DataFrame(kmeans.labels_)
# Identify the smallest cluster or the one furthest from others
# cluster_minority = clusters.value_counts()
# cluster_minority = cluster_minority[cluster_minority <= 60].index.get_level_values(0).tolist()
kmeans_labels = kmeans.predict(pca_test.iloc[:,:best_dim])
# kmeans_labels = np.isin(pred, cluster_minority).astype(int)
conf_mat = confusion_matrix(y_test,kmeans_labels)
acc = accuracy_score(y_test,kmeans_labels)
recall = recall_score(y_test,kmeans_labels)
prec = precision_score(y_test,kmeans_labels)
f1 = f1_score(y_test,kmeans_labels)
print(conf_mat)
print(f'Test accuracy: {acc:.2f}')
print(f'Test precision: {prec:.2f}')
print(f'Test recall: {recall:.2f}')
print(f'Test f1-score: {f1:.2f}')

[[225   0]
 [ 31   3]]
Test accuracy: 0.88
Test precision: 1.00
Test recall: 0.09
Test f1-score: 0.16


## KNN

In [ ]:
# Perform PCA on X_train and X_test with 6 dimensions
X_train = pca_df_inp.iloc[:, :6]  
X_test = pca_test.iloc[:, :6] 

# Train k-NN model on X_train_pca with 10 neighbors
knn = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')
knn.fit(X_train)

# Predict anomalies on X_test_pca
distances, indices = knn.kneighbors(X_test)
anomaly_scores = distances.mean(axis=1)
threshold = np.percentile(anomaly_scores, 90)

# Convert labels to binary (0: inliers, 1: outliers)
knn_labels = np.where(anomaly_scores > threshold, 1, 0)

# Evaluate the performance on X_test
conf_mat = confusion_matrix(y_test, knn_labels)
acc = accuracy_score(y_test, knn_labels)
f1 = f1_score(y_test, knn_labels)

print(f'Confusion matrix on X_test:\n{conf_mat}')
print(f'Accuracy on X_test: {acc:.2f}')
print(f'F1 Score on X_test: {f1:.2f}')
print("Predicted knn labels:")
print(knn_labels)

Confusion matrix on X_test:
[[224   1]
 [  9  25]]
Accuracy on X_test: 0.96
F1 Score on X_test: 0.83
Predicted knn labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0]


## DBSCAN

In [ ]:
# Apply PCA with 12 dimensions
X_train = pca_df_inp.iloc[:, :12]  
X_test = pca_test.iloc[:, :12] 

# Fit DBSCAN model
dbscan = DBSCAN(eps=10, min_samples=10)
dbscan_labels_train = dbscan.fit_predict(X_train)
dbscan_labels_test = dbscan.fit_predict(X_test)

# Convert labels to binary (0: inliers, 1: outliers)
dbscan_labels_train = np.where(dbscan_labels_train >= 0, 0, 1)
dbscan_labels_test = np.where(dbscan_labels_test >= 0, 0, 1)

# Evaluate performance on test data
conf_mat_test = confusion_matrix(y_test, dbscan_labels_test)
acc_test = accuracy_score(y_test, dbscan_labels_test)
f1_test = f1_score(y_test, dbscan_labels_test)

print("\nPerformance on test data:")
print("Confusion matrix:")
print(conf_mat_test)
print("Accuracy:", acc_test)
print("F1 Score:", f1_test)
print("Predicted DBScan labels:")
print(dbscan_labels_test)


Performance on test data:
Confusion matrix:
[[213  12]
 [  0  34]]
Accuracy: 0.9536679536679536
F1 Score: 0.85
Predicted DBScan labels:
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1
 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0]


## Isolation Forest

In [ ]:
#Best Model for Isolation Forest is with 10 PCA and n_estimators = 100
best_n_est = 50
pca_dim = 6
X_train = pca_df_inp.iloc[:,:pca_dim]
X_test = pca_test.iloc[:,:pca_dim]
            
isolation_forest = IsolationForest(n_estimators=best_n_est, 
                                   max_samples='auto', 
                                   random_state=42)

model = isolation_forest.fit(X_train)
if_labels = model.predict(X_test)
if_labels = np.where(if_labels == -1, 1, 0)

conf_mat   = confusion_matrix(y_test,if_labels)
acc        = accuracy_score(y_test,if_labels)
recall     = recall_score(y_test,if_labels)
prec       = precision_score(y_test,if_labels)
f1         = f1_score(y_test, if_labels)


print(f'Confusion matrix for {pca_dim} PCA dimensions:\n', 
                      f' {conf_mat}')
#print(f'Accuracy for {pca_dim} PCA dimensions:', 
#                      f' {acc:.2f}')
#print(f'Recall for {pca_dim} PCA dimensions:', 
#                      f' {recall:.2f}')
#print(f'Precision for {pca_dim} PCA dimensions:', 
#                      f' {prec:.2f}')
print(f'F1 Score for {pca_dim} PCA dimensions with n_estimator = {best_n_est}:', 
                      f' {f1:.2f}')
print(f'---------')

Confusion matrix for 6 PCA dimensions:
  [[223   2]
 [  1  33]]
F1 Score for 6 PCA dimensions with n_estimator = 50:  0.96
---------


## One-class SVM

In [ ]:
# Best Model is with 163 PCA dimensions
pca_dim = 85

X_train = pca_df_inp.iloc[:,:pca_dim]
X_test = pca_test.iloc[:,:pca_dim]

oneclass = OneClassSVM(gamma = 'auto').fit(X_train)
oneclass_labels = oneclass.predict(X_test)
    
oneclass_labels = np.where(oneclass_labels == 1, 0,1)
conf_mat   = confusion_matrix(y_test,oneclass_labels)
acc        = accuracy_score(y_test,oneclass_labels)
recall     = recall_score(y_test,oneclass_labels)
prec       = precision_score(y_test,oneclass_labels)
f1         = f1_score(y_test,oneclass_labels)

print(f'Confusion matrix for {pca_dim} PCA dimensions:\n', 
              f' {conf_mat}')
#print(f'Accuracy for {pca_dim} PCA dimensions:', 
#              f' {acc:.2f}')
#print(f'Recall for {pca_dim} PCA dimensions:', 
#              f' {recall:.2f}')
#print(f'Precision for {pca_dim} PCA dimensions:', 
#              f' {prec:.2f}')
print(f'F1 Score for {pca_dim} PCA dimensions:', 
              f' {f1:.2f}')

Confusion matrix for 85 PCA dimensions:
  [[ 69 156]
 [  0  34]]
F1 Score for 85 PCA dimensions:  0.30


## Combine

In [ ]:
best_acc = -float('inf')
best_dim = -1
best_recall = -1
best_prec = -1
best_f1 = -1

for i1 in [0,1]:
    for i2 in [0,1]:
        for i3 in [0,1]:
            for i4 in [0,1]:
                for i5 in [0,1]:
                    for thresh in [0,1]:
                        tot = i1+i2+i3+i4+i5
                        comb_labels = i1*if_labels + i2*oneclass_labels + i3*dbscan_labels_test + \
                                        i4*kmeans_labels + i5*knn_labels
                        comb_labels = np.where(comb_labels > tot//2 + thresh, 1, 0)

                        conf_mat   = confusion_matrix(y_test,comb_labels)
                        acc        = accuracy_score(y_test,comb_labels)
                        recall     = recall_score(y_test,comb_labels)
                        prec       = precision_score(y_test,comb_labels)
                        f1         = f1_score(y_test,comb_labels)

                        if acc > best_acc:
                            best_acc = acc
                            best_recall  = recall
                            best_prec = prec
                            best_conf_mat = conf_mat
                            best_f1 = f1
print(f'Confusion matrix for ({i1,i2,i3,i4,i5}) included and {thresh}:\n', f' {conf_mat}')
print(f'Best f1 score: {best_f1:.2f}')
                        #     print(f'Accuracy for ({i1,i2,i3,i4,i5}) included and {thresh}:', 
                        #           f' {acc:.2f}')
                        #     print(f'Recall for ({i1,i2,i3,i4,i5}) included and {thresh}:', 
                        #           f' {recall:.2f}')
                        #     print(f'Precision for ({i1,i2,i3,i4,i5}) included and {thresh}:', 
                        #           f' {prec:.2f}')

Confusion matrix for ((1, 1, 1, 1, 1)) included and 1:
  [[224   1]
 [ 10  24]]
Best f1 score: 0.97
